In [118]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image

In [31]:
DATASET_PATH ="/home/samarthramesh/Work/Build-NNs/data/face_data/lfw-deepfunneled/lfw-deepfunneled/"

In [55]:
dataset = {"person" : [], "path" : []}
for path in glob.iglob(os.path.join(DATASET_PATH, "**", "*.jpg")):
    person = path.split("/")[-2]
    dataset["person"].append(person)
    dataset["path"].append(path)
    
dataset = pd.DataFrame(dataset)
# remove people with too many images
# todo: randomly select which images to keep and discard excess
dataset = dataset.groupby("person").filter(lambda x: len(x) < 25 )
dataset.head(10)

,person,path
0,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
1,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
2,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
3,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
4,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
5,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
6,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
7,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
8,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...
9,Abdullah_Gul,/home/samarthramesh/Work/Build-NNs/data/face_d...


In [63]:
class FaceDataset(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df
    
    def __len__(self):
        return self.data_df.shape[0]
    
    def __getitem__(self, index):
        data_point = self.data_df.iloc[index]
        name = data_point["person"]
        face = read_image(data_point["path"])
        return face, name

In [64]:
train_dataloader = DataLoader(FaceDataset(dataset), batch_size=32, shuffle=True)

In [107]:
inputs, labels = next(iter(train_dataloader))
inputs = inputs/255

In [70]:
type(inputs)

torch.Tensor

In [68]:
len(labels)

32

In [69]:
labels

('Jennifer_Keller',
 'Ajit_Agarkar',
 'Catherine_Deneuve',
 'Jennifer_Aniston',
 'Martin_Bandier',
 'Al_Leiter',
 'Peter_OToole',
 'Bruce_Van_De_Velde',
 'Matt_Dillon',
 'Tali_Imani',
 'Zhu_Rongji',
 'Katie_Harman',
 'Robert_Duvall',
 'Bono',
 'Saddam_Hussein',
 'Florencia_Kirchner',
 'Rachel_Hunter',
 'King_Gyanendra',
 'Hank_Aaron',
 'Jimmy_Lee',
 'Winona_Ryder',
 'Dale_Earnhardt_Jr',
 'Paul_Murphy',
 'Dave_Barr',
 'Ronde_Barber',
 'Roman_Abramovich',
 'Keanu_Reeves',
 'Anna_Kournikova',
 'Audrey_Sauret',
 'Eli_Rosenbaum',
 'Hootie_Johnson',
 'Natalie_Imbruglia')

In [76]:
def calc_output_res(in_channels, num_layers, kernel_size=3, stride=2, padding=1):
    for _ in range(num_layers):
        in_channels = np.floor(((in_channels-kernel_size+2*padding)/stride) + 1)
    return int(in_channels)

class VAE(nn.Module):
    def __init__(self, in_channels, latent_dim, hidden_dims = None, kernel_size=3, stride=2, padding=1):
        super().__init__()
        self.latent_dim = latent_dim
        self.hidden_dims = hidden_dims

        if self.hidden_dims is None:
            self.hidden_dims = [32, 64, 128, 256]

        encoder_modules = []
        for dim in range(len(self.hidden_dims) - 1):
            encoder_modules.append(
                nn.Sequential(
                    nn.Conv2d(
                        in_channels,
                        dim,
                        kernel_size=kernel_size,
                        stride=stride,
                        padding=padding
                    ),
                    nn.BatchNorm2d(dim),
                    nn.ReLU()
                )
            )
            in_channels = dim


        self.encoder_output_res = calc_output_res(in_channels, len(self.hidden_dims), kernel_size, stride, padding)
        if self.encoder_output_res <= 0:
            raise ValueError("Too Many Convolution Layers. Output Image is reduced to nothing.")

        self.encoder = nn.Sequential(*encoder_modules)
        self.fc_mean = nn.Linear(self.hidden_dims[-1]*(self.encoder_output_res**2), self.latent_dim)
        self.fc_vars = nn.Linear(self.hidden_dims[-1]*(self.encoder_output_res**2), self.latent_dim)

        reversed_dims = list(reversed(self.hidden_dims))
        decoder_modules = []
        for i in range(len(reversed_dims)-1):
            decoder_modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(
                        reversed_dims[i],
                        reversed_dims[i+1],
                        kernel_size=kernel_size,
                        stride=stride,
                        padding=padding,
                        output_padding=padding
                    ),
                    nn.BatchNorm2d(reversed_dims[i+1]),
                    nn.ReLU()
                )
            )

        self.decoder = nn.Sequential(*decoder_modules)

        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(
                self.hidden_dims[-1],
                self.hidden_dims[-1],
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                output_padding=padding
            ),
            nn.BatchNorm2d(self.hidden_dims[-1]),
            nn.ReLU(),
            nn.Conv2d(
                hidden_dims[-1],
                3,
                kernel_size=3,
                padding=1
            ),
            nn.Sigmoid()
        )

    def encode(self, x):
        

        
    #     self.conv_layers = [
    #         nn.Conv2d(hidden_dims[i], hidden_dims[i+1],
    #                   kernel_size=3,
    #                   stride=2,
    #                   padding=1)
    #                   for i in range(len(hidden_dims)-1)
    #     ]

    #     self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
    #     self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
    #     self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
    #     self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)

    #     self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
    #     self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
    #     self.deconv3 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
    #     self.deconv4 = nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1)

    # def encoder(self, x):
    #     x = F.relu(self.conv1(x))
    #     x = F.relu(self.conv2(x))
    #     x = F.relu(self.conv3(x))
    #     x = F.relu(self.conv4(x))
    #     x = x.view(-1, 256*14*14)


torch.nn.modules.module.Module

In [121]:
thing = [1,2,3]
thing.reverse()

In [126]:
thing2 = list(reversed(thing))

In [127]:
thing2

[3, 2, 1]

In [129]:
input_size = 250
kernel_size = 3
padding = 0
stride = 2
n_layers = 10

print(input_size)
for i in range(n_layers):
    input_size = np.floor(((input_size-kernel_size+2*padding)/stride) + 1)
    print(input_size)

# print((((input_size-kernel_size+2*padding)/stride) + 1))
# print(np.floor(((input_size-kernel_size+2*padding)/stride) + 1))

250
124.0
61.0
30.0
14.0
6.0
2.0
0.0
-1.0
-1.0
-1.0


In [108]:
conv = nn.Conv2d(3, 16, 3, stride=2)
conv(inputs).shape

torch.Size([32, 16, 124, 124])

In [106]:
inputs.dtype

torch.float32

In [102]:
conv.

Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))

In [105]:
inputs.unique()

tensor([0.0000, 0.0039, 0.0078, 0.0118, 0.0157, 0.0196, 0.0235, 0.0275, 0.0314,
        0.0353, 0.0392, 0.0431, 0.0471, 0.0510, 0.0549, 0.0588, 0.0627, 0.0667,
        0.0706, 0.0745, 0.0784, 0.0824, 0.0863, 0.0902, 0.0941, 0.0980, 0.1020,
        0.1059, 0.1098, 0.1137, 0.1176, 0.1216, 0.1255, 0.1294, 0.1333, 0.1373,
        0.1412, 0.1451, 0.1490, 0.1529, 0.1569, 0.1608, 0.1647, 0.1686, 0.1725,
        0.1765, 0.1804, 0.1843, 0.1882, 0.1922, 0.1961, 0.2000, 0.2039, 0.2078,
        0.2118, 0.2157, 0.2196, 0.2235, 0.2275, 0.2314, 0.2353, 0.2392, 0.2431,
        0.2471, 0.2510, 0.2549, 0.2588, 0.2627, 0.2667, 0.2706, 0.2745, 0.2784,
        0.2824, 0.2863, 0.2902, 0.2941, 0.2980, 0.3020, 0.3059, 0.3098, 0.3137,
        0.3176, 0.3216, 0.3255, 0.3294, 0.3333, 0.3373, 0.3412, 0.3451, 0.3490,
        0.3529, 0.3569, 0.3608, 0.3647, 0.3686, 0.3725, 0.3765, 0.3804, 0.3843,
        0.3882, 0.3922, 0.3961, 0.4000, 0.4039, 0.4078, 0.4118, 0.4157, 0.4196,
        0.4235, 0.4275, 0.4314, 0.4353, 

In [104]:
inputs = inputs/255